In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

X = tf.placeholder(tf.float32, shape = (None, 9), name = 'x')
y = tf.placeholder(tf.float32, shape = (None,2), name = 'y')

batch_size = 100
n_batches = int(np.ceil(712/batch_size))

def fetch_batch(batch_index):
    if batch_index < n_batches-1:
        start = batch_index * batch_size
        stop = batch_size + start
        x_batch = x_train[start:stop]
        y_batch = y_train[start:stop]
        
    else:
        start = batch_index* batch_size
        x_batch = x_train[start:]
        y_batch = y_train[start:]
    return x_batch, y_batch

a,b = fetch_batch(7)

In [ ]:
n_inputs = 9
h1 = 100
h2 = 300
n_output = 2

def neuron_layer(X, n_neurons, name, activation = None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/ np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev = stddev)
        W = tf.Variable(init, name = 'weights')
        b = tf.Variable(tf.zeros([n_neurons]), name = 'biases')
        z = tf.matmul(X, W) + b
        
        if activation == 'relu':
            return tf.nn.relu(z)
        
        else:
            return z
        
with tf.name_scope('NeuralNet'):
    hidden1 = neuron_layer(X, h1, 'hidden1', activation = 'relu')
    hidden2 = neuron_layer(hidden1, h2, 'hidden2', activation = 'relu')
    logits = neuron_layer(hidden2, n_output, 'outputs')

In [ ]:
with tf.name_scope('precision'):
    #fn, fp, tn, tp
    
    prediction = tf.argmax(logits, 1)
    actual = tf.argmax(y,1)
    
    TP = tf.math.count_nonzero(prediction * actual)
    TN = tf.math.count_nonzero((prediction - 1) * (actual - 1))
    FP = tf.math.count_nonzero(prediction * (actual - 1))
    FN = tf.math.count_nonzero((prediction - 1) * actual)
    
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    precision = TP/(TP+FP)
    Recall = TP/(TP+FN)
    F1_Score = 2*(Recall * precision) / (Recall + precision)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [ ]:
def one_hot(y):
    frame = pd.DataFrame({'k':y.astype(np.float32)})
    return pd.get_dummies(frame, columns = ['k'])

In [ ]:
n_epochs = 20
batch_size = 50
with tf.Session() as sess:
    init.run()
    max_acc=0
    acc_going_down=0
    for epoch in range(n_epochs):
        batch_step=0
        avg_loss = 0
        total_loss= 0
        total_batch = int(x_train.shape[0]/batch_size)

        for batch_index in range(total_batch):
            x_batch, y_batch = fetch_batch(batch_index)
            _,l=sess.run([training_op,loss],feed_dict={X:x_batch, y:one_hot(y_batch)})
            batch_step+=1
            total_loss += l
        if((epoch)%10==0):
            avg_loss = total_loss/batch_size
            print("Epoch:", '%02d' % (epoch+1), "| Average Training Loss= {:.2f}".format(avg_loss), 
                  "| F1_train_score:  {:.2f}".format(F1_Score.eval({X: x_train, y: one_hot(y_train)})),
                  "| F1_test_score:  {:.2f}".format(F1_Score.eval({X: x_test, y: one_hot(y_test)})))
    print("Model fit complete.")
    print("Final Training f1_score: {:.2f}".format(F1_Score.eval({X: x_train, y: one_hot(y_train)})))
    print("Final Validation f1_score: {:.2f}".format(F1_Score.eval({X: x_test, y: one_hot(y_test)})))

In [ ]:
from keras.backend import clear_session
clear_session()
from tensorflow.compat.v1 import keras
from tensorflow.keras import layers

import datetime
logdir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model = tf.keras.Sequential()
model.add(layers.Dense(400,input_shape=(9,), activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(2, activation = 'softmax'))

# model compilation
model.compile(optimizer=tf.train.GradientDescentOptimizer(learning_rate = 0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# fitting the model
from keras.utils import to_categorical
y_binary = to_categorical(y_train)
y_k = to_categorical(y_test)
h = model.fit(x_train, y_binary, epochs=50, batch_size=50,
              validation_data=(x_test, y_k),
              callbacks=[tensorboard_callback])
view rawkeras_tensors.py hosted with ❤ by GitHub
import datetime, os
import tensorboard
%load_ext tensorboard

#%reload_ext tensorboard
logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir} --host localhost --port 8008